In [1]:
import os
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
# Define LLM to be used
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [56]:
# Import the required LangChain modules
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

In [57]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Step 1 - Document Ingestion

In [41]:
### Read the list of PDFs in the dir

from langchain_community.document_loaders import PyMuPDFLoader
import os

pdfs = []
for root, dirs, files in os.walk("Scripts"):
    # print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

In [42]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)

In [43]:
len(docs)

2

In [44]:
docs

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-09-23T17:24:55+05:30', 'source': 'Scripts\\Dummy_Script.pdf', 'file_path': 'Scripts\\Dummy_Script.pdf', 'total_pages': 2, 'format': 'PDF 1.7', 'title': '', 'author': 'Amit Bhatia', 'subject': '', 'keywords': '', 'moddate': '2025-09-23T17:24:55+05:30', 'trapped': '', 'modDate': "D:20250923172455+05'30'", 'creationDate': "D:20250923172455+05'30'", 'page': 0}, page_content="/* Define a library for raw data if not already defined */ \nlibname rawdata '/sas/data/raw'; \n \n/* Define a library for processed data if not already defined */ \nlibname processed '/sas/data/processed'; \n \n/* Step 1: Load raw customer orders data */ \ndata processed.customer_orders_raw; \n    set rawdata.orders; \n    /* Simulate adding a new derived variable */ \n    OrderValue = Quantity * UnitPrice; \nrun; \n \n/* Step 2: Filter orders for a specific region */ \ndata processed.customer_orders_filter

In [45]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = text_splitter.split_documents(docs)

In [46]:
vector_store = FAISS.from_documents(chunks, embeddings)

In [7]:
db_name = r"C:\Users\91852\Documents\Machine Learning\Gen AI\AI_Agents\RAG_Data_Lineage\Scripts"
vector_store = FAISS.load_local(db_name, embeddings, allow_dangerous_deserialization=True)

#db_name = r"C:\Users\91852\Documents\Machine Learning\Gen AI\Ollama_Udemy\9.Vector_Stores\health_supplements"
#vector_store = FAISS.load_local(db_name, embeddings, allow_dangerous_deserialization=True)

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open C:\Users\91852\Documents\Machine Learning\Gen AI\AI_Agents\RAG_Data_Lineage\Scripts\index.faiss for reading: No such file or directory

Step 1b - Document Splitting

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
chunks = splitter.create_documents([transcript])

In [12]:
len(chunks)

168

In [13]:
chunks[12]

Document(metadata={}, page_content="i mean all machines do that to some extent they all enhance our natural capabilities obviously cars make us allow us to move faster than we can run but this was a machine to extend the mind and and then of course ai is the ultimate expression of what a machine may be able to do or learn so very naturally for me that thought extended into into ai quite quickly remember the the programming language that was first started special to the machine no it was just the base it was just i think it was just basic uh on the zx spectrum i don't know what specific form it was and then later on i got a commodore amiga which uh was a fantastic machine no you're just showing off so yeah well lots of my friends had atari st's and i i managed to get amigas it was a bit more powerful and uh and that was incredible and used to do um programming in assembler and and uh also amos basic this this specific form of basic it was incredible actually as well all my coding skills

Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [16]:
vector_store = FAISS.from_documents(chunks, embeddings)

IndexError: list index out of range

Step 2 - Retrieval

In [47]:
retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs={'k':4})

In [48]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FEBC66ACE0>, search_kwargs={'k': 4})

Step 3 - Augmentation

In [49]:
prompt = PromptTemplate(
    template="""
      You are a SAS code reviewer focussing on Data Lineage.
      Answer ONLY from the provided transcript context.
      Give your answer in 2 -3 sentences.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [50]:
question          = "What is the source table?"
retrieved_docs    = retriever.invoke(question)

In [51]:
retrieved_docs

[Document(id='2b3f73dc-95a1-4cad-8d69-06ffdfcd2682', metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-09-23T17:24:55+05:30', 'source': 'Scripts\\Dummy_Script.pdf', 'file_path': 'Scripts\\Dummy_Script.pdf', 'total_pages': 2, 'format': 'PDF 1.7', 'title': '', 'author': 'Amit Bhatia', 'subject': '', 'keywords': '', 'moddate': '2025-09-23T17:24:55+05:30', 'trapped': '', 'modDate': "D:20250923172455+05'30'", 'creationDate': "D:20250923172455+05'30'", 'page': 0}, page_content="/* Define a library for raw data if not already defined */ \nlibname rawdata '/sas/data/raw'; \n \n/* Define a library for processed data if not already defined */ \nlibname processed '/sas/data/processed'; \n \n/* Step 1: Load raw customer orders data */ \ndata processed.customer_orders_raw; \n    set rawdata.orders; \n    /* Simulate adding a new derived variable */ \n    OrderValue = Quantity * UnitPrice; \nrun; \n \n/* Step 2: Filter orders for a specific region

In [52]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"/* Define a library for raw data if not already defined */ \nlibname rawdata '/sas/data/raw'; \n \n/* Define a library for processed data if not already defined */ \nlibname processed '/sas/data/processed'; \n \n/* Step 1: Load raw customer orders data */ \ndata processed.customer_orders_raw; \n    set rawdata.orders; \n    /* Simulate adding a new derived variable */ \n    OrderValue = Quantity * UnitPrice; \nrun; \n \n/* Step 2: Filter orders for a specific region */ \ndata processed.customer_orders_filtered; \n    set processed.customer_orders_raw; \n    where Region = 'East'; \nrun; \n \n/* Step 3: Aggregate sales by customer */ \nproc sql; \n    create table processed.customer_sales_summary as \n    select \n        CustomerID, \n        sum(OrderValue) as TotalSales, \n        count(distinct OrderID) as NumberOfOrders \n    from processed.customer_orders_filtered \n    group by CustomerID; \nquit; \n \n/* Step 4: Merge aggregated sales with customer demographic information */\n\

In [53]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [54]:
final_prompt

StringPromptValue(text="\n      You are a SAS code reviewer focussing on Data Lineage.\n      Answer ONLY from the provided transcript context.\n      Give your answer in 2 -3 sentences.\n      If the context is insufficient, just say you don't know.\n\n      /* Define a library for raw data if not already defined */ \nlibname rawdata '/sas/data/raw'; \n \n/* Define a library for processed data if not already defined */ \nlibname processed '/sas/data/processed'; \n \n/* Step 1: Load raw customer orders data */ \ndata processed.customer_orders_raw; \n    set rawdata.orders; \n    /* Simulate adding a new derived variable */ \n    OrderValue = Quantity * UnitPrice; \nrun; \n \n/* Step 2: Filter orders for a specific region */ \ndata processed.customer_orders_filtered; \n    set processed.customer_orders_raw; \n    where Region = 'East'; \nrun; \n \n/* Step 3: Aggregate sales by customer */ \nproc sql; \n    create table processed.customer_sales_summary as \n    select \n        CustomerI

In [55]:
answer = llm.invoke(final_prompt)
print(answer.content)

Based on the provided code, there are two primary source tables from the `rawdata` library. The `rawdata.orders` table is used to create the initial customer orders dataset, and the `rawdata.customer_demographics` table is merged in the final step.


Building a Chain

In [58]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [59]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [60]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
    }
)

In [61]:
parallel_chain.invoke("What child tables are created from the table 'SALES' ?")

{'context': "/* Define a library for raw data if not already defined */ \nlibname rawdata '/sas/data/raw'; \n \n/* Define a library for processed data if not already defined */ \nlibname processed '/sas/data/processed'; \n \n/* Step 1: Load raw customer orders data */ \ndata processed.customer_orders_raw; \n    set rawdata.orders; \n    /* Simulate adding a new derived variable */ \n    OrderValue = Quantity * UnitPrice; \nrun; \n \n/* Step 2: Filter orders for a specific region */ \ndata processed.customer_orders_filtered; \n    set processed.customer_orders_raw; \n    where Region = 'East'; \nrun; \n \n/* Step 3: Aggregate sales by customer */ \nproc sql; \n    create table processed.customer_sales_summary as \n    select \n        CustomerID, \n        sum(OrderValue) as TotalSales, \n        count(distinct OrderID) as NumberOfOrders \n    from processed.customer_orders_filtered \n    group by CustomerID; \nquit; \n \n/* Step 4: Merge aggregated sales with customer demographic infor

In [62]:
parser = StrOutputParser()

In [63]:
main_chain = parallel_chain | prompt | llm | parser

In [72]:
question = "What all will break if 'customer_orders_filtered' is deleted ?"

In [73]:
main_chain.invoke(question)

'Based on the provided code, deleting `customer_orders_filtered` will cause the `proc sql` step (Step 3) to fail because it cannot create the `customer_sales_summary` table. This failure will then cause the final data merge step (Step 4) to also fail, as it depends on the `customer_sales_summary` table.'

In [74]:
question2 = "What will break if column 'quantity' is deleted from the table 'orders' ?"

In [75]:
main_chain.invoke(question2)

"Based on the provided code, deleting the 'quantity' column from the 'orders' table will cause the first data step to fail. This step fails because it tries to calculate the `OrderValue` by multiplying `Quantity` and `UnitPrice`. This error will prevent the creation of `processed.customer_orders_raw` and cause all subsequent processing steps to fail as well."